## 자영업자는 농수축산물 일자별 도소매 가격으로 어떤 인사이트를 얻을 수 있을까요?

* 가격 변동 원인: 특정 품목의 가격이 연중 어느 시기에 가장 높고 낮은지, 그 원인은 무엇일까요?
* 지역 차이 요인: 어떤 지역에서 특정 품목의 가격이 가장 높고 낮은지, 그 이유는 무엇일까요?
* 최적 할인 전략: 어떤 품목에서 할인 정책이 가장 효과적이며, 최적의 할인 전략(할인 폭, 기간 등)은 무엇일까요?
* 품종/등급 선호도: 어떤 품종이 가장 높은 가격 프리미엄을 가지며, 소비자들의 품종 및 등급 선호도는 어떻게 변화하고 있을까요?
* 유통 효율화 방안: 어떤 품목에서 도소매 가격 차이가 가장 크고 작은지, 유통 마진을 줄이기 위한 효과적인 방안은 무엇일까요?

## 농수축산물 일자별 도소매 가격

https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_6faffdb1-c1c0-4c1e-aacb-9eb34b32f772/5

In [ ]:
# !pip install -Uq koreanize-matplotlib

In [ ]:
import pandas as pd
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# 데이터 컬럼 정보를 불러옵니다.|
df_col = pd.read_csv("data/kadx-utf-8-at-product-price-detail.csv")
df_col.shape

In [ ]:
df_col

In [ ]:
# 원본 데이터를 불러옵니다.
raw = pd.read_csv("data/kadx-utf-8-at-product-price.csv")
raw.shape

In [ ]:
# 영문 컬럼명을 한글로 변환하기 위한 매핑 딕셔너리를 생성합니다.
col_dict = df_col[["nameEn","nameKo"]].set_index("nameEn")['nameKo'].to_dict()
col_dict

In [ ]:
# 원본 데이터의 컬럼명을 한글로 변환합니다.
df = raw.rename(columns=col_dict)

In [ ]:
# 변환된 데이터의 첫 몇 줄을 확인합니다.
df.head()

In [ ]:
df.columns

## 기초 기술통계

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(exclude="number")

### 유일값 분석

In [ ]:
df.nunique()

In [ ]:
nuniq = df.nunique() 
nuniq[nuniq > 1]

## 데이터 타입 변경

In [ ]:
df["가격등록일자"] = pd.to_datetime(df["가격등록일자"].astype(str))

## 빈도 분석

In [ ]:
df["품종명"].value_counts()

## 범주별 가격 분석

In [ ]:
plt.figure(figsize=(8, 3))
sns.barplot(data=df, x='산물등급명', y='품목가격')
plt.show()

In [ ]:
plt.figure(figsize=(16, 3))
sns.barplot(data=df, x='품목명', y='품목가격')
plt.show()

In [ ]:
plt.figure(figsize=(16, 3))
sns.barplot(data=df, x='품목명', y='품목가격', hue="할인가격여부")
plt.show()

In [ ]:
pd.crosstab(df["할인가격여부"], df["품목명"])

In [ ]:
df.groupby(["품목명", "할인가격여부"])["품목가격"].mean().unstack().style.format("{:,.0f}")

In [ ]:
plt.figure(figsize=(8, 3))
sns.barplot(data=df, x='산물등급명', y='품목가격')
plt.show()

In [ ]:
plt.figure(figsize=(8, 3))
sns.barplot(data=df, x='산물등급명', y='품목가격', hue="할인가격여부")
plt.show()

## 상관 분석

In [ ]:
corr = df.corr(numeric_only=True).dropna(axis=0, how="all").dropna(axis=1, how="all")
corr

In [ ]:
mask = np.triu(np.ones_like(corr))

In [ ]:
sns.heatmap(corr, annot=True, fmt=".2f", mask=mask, cmap="seismic")

### 상관 관계 분석을 통한 인사이트 도출

주어진 상관 행렬을 통해 각 변수 간의 상관 관계를 분석할 수 있습니다. 상관 계수는 -1에서 1 사이의 값을 가지며, 1에 가까울수록 강한 양의 상관 관계를, -1에 가까울수록 강한 음의 상관 관계를 의미합니다. 0에 가까울수록 두 변수 간의 상관 관계가 거의 없음을 나타냅니다.

### 주요 상관 관계 분석:

1. **도매출하단위크기와 품목코드**:
   - **상관 계수**: -0.929279
   - **해석**: 매우 강한 음의 상관 관계가 있습니다. 이는 도매출하단위크기가 클수록 품목코드가 낮다는 것을 의미합니다. 품목코드는 일반적으로 특정 품목을 나타내는 코드이므로, 도매출하단위크기가 큰 품목일수록 특정한 코드 범위 내에 있다는 점을 시사할 수 있습니다.

2. **소매출하단위크기와 품목코드**:
   - **상관 계수**: -0.651338
   - **해석**: 강한 음의 상관 관계를 보입니다. 도매출하단위크기와 유사하게, 소매출하단위크기 또한 품목코드와 반비례하는 경향이 있습니다. 이는 소매 단위가 클수록 특정 품목코드와 연관되어 있음을 나타냅니다.

3. **품종코드와 품목가격**:
   - **상관 계수**: 0.595948
   - **해석**: 양의 상관 관계가 있습니다. 이는 특정 품종코드가 품목가격과 상당한 관련이 있음을 나타냅니다. 즉, 특정 품종은 가격이 더 높거나 낮을 가능성이 있습니다.

4. **산물등급코드와 품종코드**:
   - **상관 계수**: 0.532070
   - **해석**: 중간 정도의 양의 상관 관계를 보입니다. 이는 특정 산물 등급이 특정 품종과 자주 연관되어 있음을 시사합니다. 예를 들어, 고급 산물 등급은 특정 품종에서 주로 나타날 수 있습니다.

5. **도매출하단위크기와 소매출하단위크기**:
   - **상관 계수**: 0.618288
   - **해석**: 강한 양의 상관 관계를 나타냅니다. 도매출하단위크기가 클수록 소매출하단위크기도 큰 경향이 있습니다. 이는 도매와 소매의 단위 크기가 함께 증가하는 경향이 있음을 보여줍니다.

### 비즈니스적 인사이트:

1. **제품 분류와 단위 크기**:
   - 도매와 소매의 단위 크기는 품목코드와 강한 음의 상관 관계를 보입니다. 이는 특정 품목코드(예: 대량 유통되는 상품)와 관련된 제품이 도매와 소매 단위 크기가 큰 경향이 있다는 것을 나타냅니다. 자영업자나 유통업체는 특정 품목에 대해 대량 구매가 필요한지, 혹은 소량 구매가 더 적합한지 판단할 수 있습니다.

2. **품종에 따른 가격 전략**:
   - 품종코드와 품목가격 간의 양의 상관 관계는 특정 품종이 다른 품종보다 더 높은 가격을 받을 수 있음을 시사합니다. 이는 가격 전략 수립 시 특정 품종에 대해 더 높은 가격을 책정하거나, 고급화를 통해 부가가치를 높이는 전략을 고려할 수 있습니다.

3. **품종과 산물 등급의 연관성**:
   - 특정 품종이 특정 산물 등급과 자주 연관된다는 점에서, 품질 관리 및 마케팅에 있어 특정 품종을 고급 등급으로 브랜딩하는 것이 중요할 수 있습니다. 예를 들어, 특정 품종은 고급 등급으로 분류되어 프리미엄 가격으로 판매될 수 있습니다.

4. **도매와 소매의 일관된 단위 크기**:
   - 도매와 소매 단위 크기의 강한 상관 관계는 공급망 관리에서 도매 및 소매의 단위 크기 조정이 필요할 때 일관된 접근이 필요함을 의미합니다. 이는 공급망 효율성을 높이고, 재고 관리의 일관성을 유지하는 데 도움이 될 수 있습니다.

5. **가격과 단위 크기 조정**:
   - 품목가격과 도매/소매 단위 크기 간의 음의 상관 관계를 통해, 단위 크기를 조정하여 가격을 최적화하는 전략을 고려할 수 있습니다. 단위 크기를 줄이면 가격을 더 높게 설정할 수 있으며, 이는 소비자의 구매 심리를 자극할 수 있습니다.


## 추가로 해볼 수 있는 것
* 시계열 분석: 월별, 계절별 가격 변동 추이를 시각화하고 품목별로 비교합니다.
* 지리적 분석: 시도별, 시군구별 평균 가격을 비교하고 지도 시각화를 수행합니다.
* 할인 효과 모델링: 할인 여부에 따른 판매량 변화를 시계열로 분석하고, 할인 폭과 판매량 증가 사이의 관계를 모델링합니다.
* 프리미엄 계산: 품종별, 등급별 평균 가격을 비교하고 프리미엄을 계산합니다.
* 유통 마진 분석: 도매가와 소매가의 차이를 품목별, 시기별로 비교하고, 유통 단계별 가격 변동의 시차 효과를 분석합니다.